# Assignment 2

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import abc
from sklearn.model_selection import train_test_split

In [ ]:
np.random.seed(30)

## Task 1

**Description**
Apply the logistic regression method using the functions in the classwork notebook to predict biological response of a molecule. Data (bioresponse) from Kaggle

**Helper functions from classwork**

In [ ]:
def sigmoid(z):
    return 1. / (1. + np.exp(-z))

In [ ]:
def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0.

    return w, b

**Forward and backward propagation**

In [ ]:
def propagate(w, b, X, y):

    m = X.shape[1]
    #print('number of objects = ',len(X))
    
    # FORWARD PROPAGATION (FROM X TO COST)
    a = sigmoid(np.dot(w.T, X) + b)
    cost = -(1. / m) * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a), axis = 1)
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (1. / m) * np.dot(X, (a - y).T)
    db = (1. / m) * np.sum(a - y, axis = 1)

    grads = {"dw": dw, "db": db}
    
    return grads, cost

In [ ]:
w, b, X, y = np.array([[1.],[-1.]]), 4., np.array([[1.,5.,-1.],[10.,0.,-3.2]]), np.array([[0,1,1]])
grads, cost = propagate(w, b, X, y)
print("dw = " + str(grads["dw"]))
print("db = " + str(grads["db"]))
print("cost = " + str(cost))

**Model optimization**

In [ ]:
def optimize(w, b, X, y, num_iterations, learning_rate, print_cost = False):
    costs = []
    
    for i in range(num_iterations):
                
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule
        w -= learning_rate * dw
        b -= learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print (f"Cost after iteration {i}: {cost}")

    params = { "w": w, "b": b }
    grads = { "dw": dw, "db": db }
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, y, num_iterations= 1000, learning_rate = 0.005, print_cost = True)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

**Predictions**

In [ ]:
def predict(w, b, X):
    m = X.shape[1]
    y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)

    # Compute vector "A" predicting the probabilities
    a = sigmoid(np.dot(w.T, X) + b)

    for i in range(a.shape[1]):

        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if a[0,i] <= 0.5:
            y_pred[0][i] = 0
        else:
            y_pred[0][i] = 1

    return y_pred

In [ ]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

**Wrap into model**

In [ ]:
def model(X_train, y_train, X_test, y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

    # initialize parameters with zeros
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent
    parameters, grads, costs = optimize(w, b, X_train, y_train, num_iterations, learning_rate, print_cost)

    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]

    # Predict test/train set examples
    y_pred_test = predict(w, b, X_test)
    y_pred_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(y_pred_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_pred_test - y_test)) * 100))

    d = {
        "costs": costs,
        "y_prediction_test": y_pred_test,
        "y_prediction_train" : y_pred_train,
        "learning_rate" : learning_rate,
        "num_iterations": num_iterations,
        "w" : w,
        "b" : b,
    }

    return d

**Load data**

In [ ]:
df = pd.read_csv("bioresponse.csv")
df.head()

In [ ]:
y = df["Activity"]
X = df.drop(columns=["Activity"])

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=30, test_size=0.25)

**Running the following (using unmodified code from class work yields shape incompatibility)**

In order to have this as part of part 1 of task 1 is it here, but to make notebook actually runnable I will comment it out

In [ ]:
# model(
#     X_train, y_train.T,
#     X_test, y_test.T,
#     num_iterations = 2000,
#     learning_rate = 0.001,
#     print_cost = True
# )

## Use SGD and Adam for training

**Adam and SGD implementation**

In [ ]:
class LearningRateScheduler(abc.ABC):
    @abc.abstractmethod
    def get(self, iteration: int = None, loss: int = None) -> float:
        pass

class ConstantLearningRate(LearningRateScheduler):
    def __init__(self, lr: float = 1e-7):
        self.lr = lr

    def get(self, iteration: int = None, loss: int = None) -> float:
        return self.lr

class Optimizer(abc.ABC):
    @abc.abstractmethod
    def step(self, parameters, gradient):
        pass

    @abc.abstractmethod
    def get_batch_size(self):
        pass

class SGD(Optimizer):

    def __init__(self,
                 batch_size: int = 16,
                 lr: LearningRateScheduler = ConstantLearningRate()):
        self.batch_size = batch_size
        self.lr = lr

    def step(self, parameters, gradient):
        delta = self.lr.get() * gradient
        return delta

    def get_batch_size(self) -> int:
        return self.batch_size

class Adam(Optimizer):

    def __init__(self,
                 batch_size: int = 16,
                 beta1: float = 0.9,
                 beta2: float = 0.999,
                 eps: float = 1e-7,
                 lr: LearningRateScheduler = ConstantLearningRate()):
        self.batch_size: int = batch_size
        self.beta1: float = beta1
        self.beta2: float = beta2
        self.eps: float = eps
        self.lr: LearningRateScheduler = lr
        self.cache: dict = {
            "t": -1,
            "mean": None,
            "var": None,
        }

    def get_batch_size(self) -> int:
        return self.batch_size

    def step(self, parameters, gradient):
        if self.cache["t"] == -1:
            self.cache = {
                "t": 0,
                "mean": np.zeros_like(parameters),
                "variance": np.zeros_like(parameters)
            }

        t = self.cache["t"] + 1
        mean = self.cache["mean"]
        variance = self.cache["variance"]

        self.cache["t"] = t
        self.cache["mean"] = self.beta1 * mean + (1 - self.beta1) * gradient
        self.cache["variance"] = self.beta2 * variance + (1 - self.beta2) * gradient ** 2

        v_hat = self.cache["variance"] / (1. - (self.beta2 ** t))
        m_hat = self.cache["mean"] / (1. - (self.beta1 ** t))

        update = self.lr.get(iteration=t) * m_hat / (np.sqrt(v_hat) + self.eps)
        return update

**Helpers to define the model**

In [ ]:
def softmax(predictions):
    single = (predictions.ndim == 1)

    if single:
        predictions = predictions.reshape(1, predictions.shape[0])

    maximums = np.amax(predictions, axis=1).reshape(predictions.shape[0], 1)
    predictions_ts = predictions - maximums

    predictions_exp = np.exp(predictions_ts)
    sums = np.sum(predictions_exp, axis=1).reshape(predictions_exp.shape[0], 1)
    result = predictions_exp / sums

    if single:
        result = result.reshape(result.size)

    return result   # S

def cross_entropy_loss(probs, target_index):
    single = (probs.ndim == 1)

    if single:
        probs = probs.reshape(1, probs.shape[0])
        target_index = np.array([target_index])

    rows = np.arange(target_index.shape[0])
    cols = target_index

    return np.mean(-np.log(probs[rows, cols]))


def softmax_with_cross_entropy(predictions, target_index):
    single = (predictions.ndim == 1)

    if single:
        predictions = predictions.reshape(1, predictions.shape[0])
        target_index = np.array([target_index])

    probs = softmax(predictions)
    loss = cross_entropy_loss(probs, target_index)

    indicator = np.zeros(probs.shape)
    indicator[np.arange(probs.shape[0]), target_index] = 1
    dprediction = (probs - indicator) / predictions.shape[0]

    if single:
        dprediction = dprediction.reshape(dprediction.size)

    return loss, dprediction

def linear_softmax(X, W, target_index):
    predictions = np.dot(X, W)
    loss, dprediction = softmax_with_cross_entropy(predictions, target_index)

    dW = np.matmul(dprediction.T, X).T

    return loss, dW

In [ ]:
class LinearSoftmaxClassifier:
    def __init__(self):
        self.W = None

    def fit(self, X, y, epochs=1, verbose=10, lr=0.001, X_eval: np.array = None, y_eval: np.array = None):
        num_train = X.shape[0]
        num_features = X.shape[1]
        num_classes = np.max(y) + 1
        if self.W is None:
            self.W = 0.001 * np.random.randn(num_features, num_classes)

        loss_history = []
        for epoch in range(epochs):
            loss, dW = linear_softmax(X, self.W, y)
            self.W -= lr * dW

            if epoch % verbose  == 0:
                if X_eval is not None and y_eval is not None:
                    y_pred = self.predict(X_eval)
                    print(f"Epoch: \t{epoch},\t loss: {loss}, \t accuracy: {(y_pred == y_eval).mean()}")
                else:
                    print(f"Epoch: \t{epoch},\t loss: {loss}")

            loss_history.append(loss)

        return loss_history

    def predict(self, X):
        Z = np.dot(X, self.W)
        S = softmax(Z)

        return np.argmax(S, axis=1)
    

model = LinearSoftmaxClassifier()
history = model.fit(X_train.values, y_train.values, epochs=300, lr=0.1, X_eval=X_test, y_eval=y_test)

So we got 0.76 accuracy, which is surprising considering how simple our model is

## Task 2

**Description**

Modify the code so that it works for SGD, (gradients are evaluated in batches)

In [ ]:
class LinearSoftmaxClassifierBatched:
    def __init__(self):
        self.W = None

    def fit(self, X, y, epochs=1, verbose=10, optim = SGD(), X_eval: np.array = None, y_eval: np.array = None):
        num_train = X.shape[0]
        num_features = X.shape[1]
        num_classes = np.max(y) + 1
        if self.W is None:
            self.W = 0.001 * np.random.randn(num_features, num_classes)

        loss_history = []
        for epoch in range(epochs):
            shuffled_indices = np.arange(num_train)
            np.random.shuffle(shuffled_indices)
            batch_size = optim.get_batch_size()
            sections = np.arange(batch_size, num_train, batch_size)
            batches_indices = np.array_split(shuffled_indices, sections)

            loss = np.nan
            for batch_indices in batches_indices:
                batch_X = X[batch_indices, :]
                batch_y = y[batch_indices]

                fn_loss, fn_dW = linear_softmax(batch_X, self.W, batch_y)

                loss = fn_loss
                dW = fn_dW

                self.W -= optim.step(self.W, dW)
            
            if epoch % verbose  == 0:
                if X_eval is not None and y_eval is not None:
                    y_pred = self.predict(X_eval)
                    print(f"Epoch: \t{epoch},\t loss: {loss}, \t accuracy: {(y_pred == y_eval).mean()}")
                else:
                    print(f"Epoch: \t{epoch},\t loss: {loss}")

            loss_history.append(loss)

        return loss_history

    def predict(self, X):
        Z = np.dot(X, self.W)
        S = softmax(Z)

        return np.argmax(S, axis=1)

## GD

In [ ]:
def gd_training(lr):
    model = LinearSoftmaxClassifier()
    history = model.fit(X_train.values, y_train.values, epochs=200, lr=lr, X_eval=X_test, y_eval=y_test)
    plt.plot(history);

**lr = 0.0001**

In [ ]:
gd_training(lr=0.0001)

The curve is smooth (not surprising), both loss and accuracy are not that good so far

**lr = 0.001**

In [ ]:
gd_training(lr=0.001)

Getting better

**lr = 0.01**

In [ ]:
gd_training(lr=0.01)

Keeps getting better

**lr = 0.01**

In [ ]:
gd_training(lr=0.075)

This is as good as it gets here for GD, the other ones are not better

**lr = 0.5**

In [ ]:
gd_training(lr=0.5)

That's a fun shape, seems like we are oscillating around the local minima, but the lr scheduler is constant, so we are kinda stuck

## SGD

In [ ]:
def sgd_training(lr):
    model = LinearSoftmaxClassifierBatched()
    sgd = SGD(lr=ConstantLearningRate(lr), batch_size=64)
    history = model.fit(X_train.values, y_train.values, epochs=150, optim=sgd, X_eval=X_test, y_eval=y_test)
    plt.plot(history);

**lr = 0.0001**

In [ ]:
sgd_training(lr=0.0001)

Seems like this lr is too small, or we need to train more epochs

**lr = 0.0005**

In [ ]:
sgd_training(lr=0.0005)

Getting better for the loss, but accuracy doesn't go up, perhaps now the model tunes predictions to be closer to extremes (0 or 1), which doesn't affect accuracy metric but is important for the loss

**lr = 0.001**

In [ ]:
sgd_training(lr=0.001)

Less stable than the previous one, it gets to lower loss, but with higher variance for the loss. However, accuracy is the best we got so far

**lr = 0.005**

In [ ]:
sgd_training(lr=0.005)

**lr = 0.1**

In [ ]:
sgd_training(lr=0.1)

Actually it turns out that this lr works better then we previously picked as the best one, what happens if we increase it even more?

**lr = 0.6**

In [ ]:
sgd_training(lr=0.6)

This is for sure too high

## Adam

In [ ]:
def adam_training(lr):
    model = LinearSoftmaxClassifierBatched()
    adam = Adam(lr=ConstantLearningRate(lr), batch_size=64)
    history = model.fit(X_train.values, y_train.values, epochs=200, optim=adam, X_eval=X_test, y_eval=y_test)
    plt.plot(history);

**lr = 0.0001**

In [ ]:
adam_training(lr=0.0001)

We are getting the results better than with GD and SDG, and the accuracy and loss seems to be more stable

**Bellow are some other lr with Adam**

In general it seems that (not very surprising) in most cases Adam is better than SGD both in stability and target metric

In [ ]:
adam_training(lr=0.01)

In [ ]:
adam_training(lr=0.2)

In [ ]:
adam_training(lr=0.0001)

In [ ]:
adam_training(lr=0.0005)